**--- Part One ---**

In [396]:
## Select input data case
## Note: this assumes a plain textfile named `input_{case}` is located in the same folder as this notebook
case = "example" # <- input_example
case = "jfm"
case = "gatton"

import numpy as np
from itertools import cycle
from skimage.morphology import flood_fill

# Read input
with open(f'input_{case}','r') as f:
    input_lines = f.readlines()

faces = [np.array(i) for i in [
    [1,0,0],
    [-1,0,0],
    [0,1,0],
    [0,-1,0],
    [0,0,1],
    [0,0,-1]
]]

class Cube():
    def __init__(self,coords):
        self.coords = tuple(coords)
        self.coord = np.array(coords)
        self.x = coords[0]
        self.y = coords[1]
        self.z = coords[2]
        self.pockets = []

    def check_exposed_str(self,coords_str, grid=None, part=1):
        self.exposed = 0
        for face in faces:
            neigh = self.coord + face
            neigh_str = f'{neigh[0]},{neigh[1]},{neigh[2]}'
            # if part==1:
            if not (neigh_str in coords_str):
                if part==1:
                    self.exposed += 1
                    self.pockets.append(neigh)
                elif part==2:                
                    self.pockets.append(neigh)
                    if self.check_pocket(pocket=neigh, grid=grid):
                        self.exposed += 1

    def check_pocket(self,pocket,grid,verbose=False):
        pocket = pocket + np.array([2,2,2])
        filled_grid = flood_fill(grid,tuple(pocket),2,connectivity=1)
        if verbose: print(f'checking pocket: {pocket}')
        if filled_grid[0,0,0] == 2:
            return True
        else:
            return False

 
    def __repr__(self):
        return f'cube: ({self.x},{self.y},{self.z})'


class Grid():
    def __init__(self, part=1):
        self.cubes = []
        self.xs = []
        self.ys = []
        self.zs = []
        self.coords = []
        self.coords_str = []
        self.grid_trans = np.array([2,2,2])
        for line in input_lines:
            line=line.strip()
            coords = [int(i) for i in line.split(',')]
            self.xs.append(coords[0])
            self.ys.append(coords[1])
            self.zs.append(coords[2])
            self.coords.append(coords)
            self.coords_str.append(line)
            self.cubes.append(Cube(coords))

        if part==1:
            for cube in self.cubes:
                cube.check_exposed_str(self.coords_str,part=part)
            self.surface = sum([cube.exposed for cube in self.cubes])

        self.grid = np.zeros([max(self.xs)+6, max(self.ys)+6, max(self.zs)+6])
        for cube in self.cubes:
            self.grid[tuple(cube.coord+np.array([2,2,2]))] = 1

        if part==2:
            for cube in self.cubes:
                cube.check_exposed_str(self.coords_str, grid=self.grid, part=part)
            self.surface = sum([cube.exposed for cube in self.cubes])

In [397]:
grid = Grid()
print(grid.surface)

4548


**--- Part Two ---**

In [398]:
grid = Grid(part=2)
print(grid.surface)

2588
